<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Set data access options

In [5]:
s3options = {'anon': True,
             "client_kwargs":{
                 'endpoint_url': 'http://minio.minio:9000'
             }}

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client()

/opt/conda/lib/python3.9/site-packages/distributed/client.py:1186: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| numpy   | 1.21.0        | 1.18.1        | 1.18.1        |
| python  | 3.9.5.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [7]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('s3://sci-data/accounts/', 'accounts.00.json.gz'), storage_options=s3options)
b.take(1)

('{"id": 0, "name": "Quinn", "transactions": [{"transaction-id": 3561, "amount": 2737}, {"transaction-id": 6317, "amount": 2442}, {"transaction-id": 7323, "amount": 2634}, {"transaction-id": 8320, "amount": 2497}, {"transaction-id": 8483, "amount": 2378}, {"transaction-id": 8518, "amount": 2437}, {"transaction-id": 10236, "amount": 2510}, {"transaction-id": 11267, "amount": 2617}, {"transaction-id": 20428, "amount": 2719}, {"transaction-id": 21686, "amount": 2548}, {"transaction-id": 23576, "amount": 2374}, {"transaction-id": 32523, "amount": 2675}, {"transaction-id": 42822, "amount": 2706}, {"transaction-id": 44715, "amount": 2476}, {"transaction-id": 46333, "amount": 2538}, {"transaction-id": 47458, "amount": 2203}, {"transaction-id": 48329, "amount": 2431}, {"transaction-id": 50444, "amount": 2617}, {"transaction-id": 52627, "amount": 2508}, {"transaction-id": 55595, "amount": 2482}, {"transaction-id": 58363, "amount": 2505}, {"transaction-id": 61546, "amount": 2531}, {"transaction-

In [8]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [9]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [10]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [11]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [12]:
filenames = [os.path.join('s3://sci-data/accounts/', 'accounts.%02d.json.gz' %i) for i in range(0,50)]
lines = db.read_text(filenames, storage_options=s3options)
lines.take(3)

('{"id": 0, "name": "Quinn", "transactions": [{"transaction-id": 3561, "amount": 2737}, {"transaction-id": 6317, "amount": 2442}, {"transaction-id": 7323, "amount": 2634}, {"transaction-id": 8320, "amount": 2497}, {"transaction-id": 8483, "amount": 2378}, {"transaction-id": 8518, "amount": 2437}, {"transaction-id": 10236, "amount": 2510}, {"transaction-id": 11267, "amount": 2617}, {"transaction-id": 20428, "amount": 2719}, {"transaction-id": 21686, "amount": 2548}, {"transaction-id": 23576, "amount": 2374}, {"transaction-id": 32523, "amount": 2675}, {"transaction-id": 42822, "amount": 2706}, {"transaction-id": 44715, "amount": 2476}, {"transaction-id": 46333, "amount": 2538}, {"transaction-id": 47458, "amount": 2203}, {"transaction-id": 48329, "amount": 2431}, {"transaction-id": 50444, "amount": 2617}, {"transaction-id": 52627, "amount": 2508}, {"transaction-id": 55595, "amount": 2482}, {"transaction-id": 58363, "amount": 2505}, {"transaction-id": 61546, "amount": 2531}, {"transaction-

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [13]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Quinn',
  'transactions': [{'transaction-id': 3561, 'amount': 2737},
   {'transaction-id': 6317, 'amount': 2442},
   {'transaction-id': 7323, 'amount': 2634},
   {'transaction-id': 8320, 'amount': 2497},
   {'transaction-id': 8483, 'amount': 2378},
   {'transaction-id': 8518, 'amount': 2437},
   {'transaction-id': 10236, 'amount': 2510},
   {'transaction-id': 11267, 'amount': 2617},
   {'transaction-id': 20428, 'amount': 2719},
   {'transaction-id': 21686, 'amount': 2548},
   {'transaction-id': 23576, 'amount': 2374},
   {'transaction-id': 32523, 'amount': 2675},
   {'transaction-id': 42822, 'amount': 2706},
   {'transaction-id': 44715, 'amount': 2476},
   {'transaction-id': 46333, 'amount': 2538},
   {'transaction-id': 47458, 'amount': 2203},
   {'transaction-id': 48329, 'amount': 2431},
   {'transaction-id': 50444, 'amount': 2617},
   {'transaction-id': 52627, 'amount': 2508},
   {'transaction-id': 55595, 'amount': 2482},
   {'transaction-id': 58363, 'amount': 2

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [14]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 15,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1171, 'amount': -3520},
   {'transaction-id': 1248, 'amount': -3275},
   {'transaction-id': 2000, 'amount': -3425},
   {'transaction-id': 2707, 'amount': -3259},
   {'transaction-id': 4377, 'amount': -3243},
   {'transaction-id': 4779, 'amount': -3009},
   {'transaction-id': 4880, 'amount': -3322},
   {'transaction-id': 5142, 'amount': -3584},
   {'transaction-id': 5299, 'amount': -2793},
   {'transaction-id': 5688, 'amount': -3343},
   {'transaction-id': 5916, 'amount': -3460},
   {'transaction-id': 5976, 'amount': -2911},
   {'transaction-id': 6360, 'amount': -3197},
   {'transaction-id': 7679, 'amount': -3518},
   {'transaction-id': 8082, 'amount': -3133},
   {'transaction-id': 8748, 'amount': -3411},
   {'transaction-id': 9217, 'amount': -3506},
   {'transaction-id': 9669, 'amount': -3161},
   {'transaction-id': 10094, 'amount': -3038},
   {'transaction-id': 10255, 'amount': -2924},
   {'transaction-id': 10451, 'a

In [15]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 161},
 {'name': 'Alice', 'count': 182},
 {'name': 'Alice', 'count': 38},
 {'name': 'Alice', 'count': 94},
 {'name': 'Alice', 'count': 53})

In [16]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(161, 182, 38, 94, 53)

In [17]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

179.17142857142858

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 1171, 'amount': -3520},
  {'transaction-id': 1248, 'amount': -3275},
  {'transaction-id': 2000, 'amount': -3425},
  {'transaction-id': 2707, 'amount': -3259},
  {'transaction-id': 4377, 'amount': -3243},
  {'transaction-id': 4779, 'amount': -3009},
  {'transaction-id': 4880, 'amount': -3322},
  {'transaction-id': 5142, 'amount': -3584},
  {'transaction-id': 5299, 'amount': -2793},
  {'transaction-id': 5688, 'amount': -3343},
  {'transaction-id': 5916, 'amount': -3460},
  {'transaction-id': 5976, 'amount': -2911},
  {'transaction-id': 6360, 'amount': -3197},
  {'transaction-id': 7679, 'amount': -3518},
  {'transaction-id': 8082, 'amount': -3133},
  {'transaction-id': 8748, 'amount': -3411},
  {'transaction-id': 9217, 'amount': -3506},
  {'transaction-id': 9669, 'amount': -3161},
  {'transaction-id': 10094, 'amount': -3038},
  {'transaction-id': 10255, 'amount': -2924},
  {'transaction-id': 10451, 'amount': -3546},
  {'transaction-id': 11298, 'amount': -3093},
  {'tr

In [19]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1171, 'amount': -3520},
 {'transaction-id': 1248, 'amount': -3275},
 {'transaction-id': 2000, 'amount': -3425})

In [20]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(-3520, -3275, -3425)

In [21]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1436.662685377133

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [22]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [23]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [24]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [25]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [26]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 252), ('Alice', 252), ('Alice', 273), ('Alice', 273), ('Bob', 156), ('Bob', 156), ('Bob', 169), ('Bob', 169), ('Charlie', 382), ('Charlie', 383), ('Charlie', 414), ('Charlie', 414), ('Dan', 108), ('Dan', 108), ('Dan', 117), ('Dan', 117), ('Edith', 192), ('Edith', 192), ('Edith', 208), ('Edith', 208), ('Frank', 180), ('Frank', 180), ('Frank', 195), ('Frank', 195), ('George', 180), ('George', 180), ('George', 195), ('George', 195), ('Hannah', 228), ('Hannah', 228), ('Hannah', 247), ('Hannah', 247), ('Ingrid', 240), ('Ingrid', 240), ('Ingrid', 260), ('Ingrid', 260), ('Jerry', 261), ('Jerry', 264), ('Jerry', 285), ('Jerry', 286), ('Kevin', 228), ('Kevin', 228), ('Kevin', 247), ('Kevin', 247), ('Laura', 180), ('Laura', 180), ('Laura', 195), ('Laura', 195), ('Michael', 216), ('Michael', 216), ('Michael', 234), ('Michael', 234), ('Norbert', 263), ('Norbert', 264), ('Norbert', 285), ('Norbert', 286), ('Oliver', 276), ('Oliver', 276), ('Oliver', 299), ('Oliver', 299), ('Patricia', 22

In [27]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1050), ('Bob', 650), ('Charlie', 1593), ('Dan', 450), ('Edith', 800), ('Frank', 750), ('George', 750), ('Hannah', 950), ('Ingrid', 1000), ('Jerry', 1096), ('Kevin', 950), ('Laura', 750), ('Michael', 900), ('Norbert', 1098), ('Oliver', 1150), ('Patricia', 950), ('Quinn', 1150), ('Ray', 890), ('Sarah', 900), ('Tim', 1399), ('Ursula', 910), ('Victor', 1000), ('Wendy', 934), ('Xavier', 1050), ('Yvonne', 900), ('Zelda', 900)]
CPU times: user 29.2 ms, sys: 1.16 ms, total: 30.3 ms
Wall time: 2.83 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [ ]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [28]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Quinn,"[{'transaction-id': 3561, 'amount': 2737}, {'t..."
1,1,Norbert,"[{'transaction-id': 223, 'amount': 755}, {'tra..."
2,2,George,"[{'transaction-id': 68, 'amount': 982}, {'tran..."
3,3,Patricia,"[{'transaction-id': 74, 'amount': 3047}, {'tra..."
4,4,Kevin,"[{'transaction-id': 46, 'amount': 281}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [29]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 23.9 ms, sys: 3.86 ms, total: 27.8 ms
Wall time: 3.55 s


name
Alice      1050
Bob         650
Charlie    1593
Dan         450
Edith       800
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [30]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Quinn', 'amount': 2737, 'transaction-id': 3561},
 {'id': 0, 'name': 'Quinn', 'amount': 2442, 'transaction-id': 6317},
 {'id': 0, 'name': 'Quinn', 'amount': 2634, 'transaction-id': 7323})

In [31]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Quinn,2737,3561
1,0,Quinn,2442,6317
2,0,Quinn,2634,7323
3,0,Quinn,2497,8320
4,0,Quinn,2378,8483


In [32]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 33.2 ms, sys: 2.75 ms, total: 35.9 ms
Wall time: 5.12 s


name
Alice       188130
Bob         154344
Charlie     282872
Dan         125747
Edith       189027
Frank       123323
George      186715
Hannah      223218
Ingrid      204439
Jerry       244607
Kevin       162920
Laura       182457
Michael     131943
Norbert     194771
Oliver      251391
Patricia    206133
Quinn       337807
Ray         159305
Sarah       186340
Tim         212428
Ursula      155164
Victor      177272
Wendy       166312
Xavier      191933
Yvonne      160789
Zelda       200613
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [33]:
client.close()